In [1]:
from model import ML_Agent
from pso import PSO
from ga import GA
from parallel_bees import BEES
import pandas as pd
import numpy as np
from scipy import stats
from utility import split_data # (dataframe, label, do_shuffle=True, test_size=0.1):

In [2]:
df = pd.read_csv('thyroid.csv')
# thyroid.csv
df = df.sample(frac=1)
df.head(5)

,CLASS,T3RESIN,THYROXIN,THYRONINE,THYROID,TSH_VALUE
6,1,110,10.4,1.6,1.6,2.7
2,1,127,12.9,2.4,1.4,0.6
125,1,106,8.9,0.7,1.0,2.3
1,1,113,9.9,3.1,2.0,5.9
96,1,114,11.1,2.4,2.0,-0.3


In [3]:
train, validation, test = split_data(df, 'CLASS')
print(train[:5], train.shape)
print(validation[:5], validation.shape)
print(test[:5], test.shape)

   CLASS  T3RESIN  THYROXIN  THYRONINE  THYROID  TSH_VALUE
0      1      119      10.6        2.1      1.3        1.1
1      1      133       9.7        2.9      0.8        1.9
2      1      105      11.1        2.0      1.0        1.0
3      1      119       8.0        2.0      0.6        3.2
4      1      110       7.0        1.0      1.6        4.3 (172, 6)
   CLASS  T3RESIN  THYROXIN  THYRONINE  THYROID  TSH_VALUE
0      1      116      10.1        2.2      1.6        0.8
1      1      114       8.1        1.6      1.6        0.5
2      1      115      10.4        1.8      1.6        2.0
3      1      120      10.4        2.1      1.1        1.8
4      1      109       5.3        1.6      1.4        1.5 (21, 6)
   CLASS  T3RESIN  THYROXIN  THYRONINE  THYROID  TSH_VALUE
0      1      109       7.6        1.3      2.2        1.9
1      1      101       7.1        2.2      0.8        2.2
2      1      108       6.5        1.0      0.9        1.5
3      1      102       7.6        1.8 

In [4]:
def preprocessing(dataframe):
    scaled_dataframe = pd.DataFrame()
    for col in dataframe.columns:
        data_col = dataframe[col]
        scaled_dataframe[col] = stats.zscore(data_col)
    return scaled_dataframe

def one_hot_encoding(Target, n_classes):
    encoded_array = list()
    unique_label = np.unique(Target)
    rules = dict()
    for idx, label in enumerate(unique_label, 0):
        rules[label] = idx
    for target in Target:
        arr = np.zeros(n_classes)
        arr[rules[target]] = 1
        encoded_array.append(arr)
    return pd.DataFrame(np.array(encoded_array))

In [5]:
x_train, y_train = train.drop('CLASS', axis=1), train['CLASS']
x_val, y_val = validation.drop('CLASS', axis=1), validation['CLASS']
x_test, y_test = test.drop('CLASS', axis=1), test['CLASS']

In [6]:
n_class = len(np.unique(y_train))
x_train, y_train = preprocessing(x_train), one_hot_encoding(y_train, n_class)
x_val, y_val = preprocessing(x_val), one_hot_encoding(y_val, n_class)
x_test, y_test = preprocessing(x_test), one_hot_encoding(y_test, n_class)

In [7]:
bees = BEES(dict(name='pso_agent1', level='weak ai', m=100, weight_constraint=[-1.,1.], \
               c1=3, c2=0.5, c3=3., max_c1=3., min_c1=0.5, max_c2=3., min_c2=0.5, max_c3=3., min_c3=0.5, \
               velocity_constraint=[-0.05, 0.05], w=0.05, w_decay=0.99, goal=0.05, \
              initializer='uniform'))
ga = GA(dict(name='ga_agent2', level='weak weak ai', m=30, x_rate=[0.1, 0.95], m_rate=[0.05, 0.2], weight_constraint=[-1.,1.], \
            radioactive_rating=[0.1,1.0], grow_factor=[0.1, 1.0]))
model = ML_Agent(n_feature=5, target_size=3)

building object according to  {'name': 'pso_agent1', 'level': 'weak ai', 'm': 100, 'weight_constraint': [-1.0, 1.0], 'c1': 3, 'c2': 0.5, 'c3': 3.0, 'max_c1': 3.0, 'min_c1': 0.5, 'max_c2': 3.0, 'min_c2': 0.5, 'max_c3': 3.0, 'min_c3': 0.5, 'velocity_constraint': [-0.05, 0.05], 'w': 0.05, 'w_decay': 0.99, 'goal': 0.05, 'initializer': 'uniform'}


In [8]:
# add_layer(self, units, activation='relu', regularizer='l2', use_bias=True):
model.add_layer(8, 'relu', False)
model.add_layer(3, 'sigmoid', True)

In [9]:
model.compile_configuration(bees, loss='categorical_crossentropy', \
                            monitor=['accuracy', 'precision', 'recall', 'essential_metrics'], regularizer='l2') # 

In [ ]:
# x, y, max_iter=100, goal=0.1, batch_size=5, validation=False, x_val=[], y_val=[], validation_batch_size=5
minimum_loss = model.fit(x_train, y_train, max_iter=100, goal=0.001, batch_size=5, validation=True, x_val=x_val, y_val=y_val, validation_batch_size=5)
print('minimum loss:', minimum_loss)

Iteration: 0
no improvement from base 1.7976931348623157e+308

Itr: 0
C1: 0.4666666666666667, C2:0.06666666666666667, C3:0.4666666666666667
best:[2.7178698], overal:3.118369445800781
validation:[2.5464208]
Iteration: 1
no improvement from base [2.5464208]

Itr: 1
C1: 0.464524765729585, C2:0.07095046854082998, C3:0.464524765729585
best:[2.2665477], overal:2.7791018676757813
validation:[2.5464208]
Iteration: 2
no improvement from base [2.5464208]

Itr: 2
C1: 0.4623655913978495, C2:0.07526881720430109, C3:0.4623655913978495
best:[1.9851439], overal:2.899244384765625
validation:[2.3468897]
Iteration: 3
no improvement from base [2.3468897]

Itr: 3
C1: 0.4601889338731444, C2:0.07962213225371119, C3:0.4601889338731444
best:[1.6777992], overal:4.225523376464844
validation:[1.8910419]
Iteration: 4
no improvement from base [1.8910419]

Itr: 4
C1: 0.45799457994579945, C2:0.08401084010840108, C3:0.45799457994579945
best:[1.0344936], overal:2.481830749511719
validation:[1.7799964]
Iteration: 5


In [ ]:
loss, board = model._evaluate(x_test, y_test, len(y_test))
print('loss', loss)
print(board)
y_pred = model._predict(x_test)
y_test = np.array(y_test)
loss = model.__loss__(y_pred, y_test)
print(loss)

In [ ]:
from metrics import MSE
mse_loss = MSE(y_pred, y_test)
print(mse_loss)

In [ ]:
for name, met in board.items():
    if type(met) == dict:
        print(name)
        for key, value in met.items():
            print(key, value)
    else:
        print(name, met)

In [ ]:
i = 0
for pred, true in zip(y_pred, y_test):
    print(pred, 'VS', true)
    i += 1

In [ ]:
for idx, particle in enumerate(bees.swams):
    print(idx, particle.fitness)

In [ ]:
wt = model.flat_weight
for w in wt:
    print(w)

In [ ]:
layer_weight = model.weight_matrix
layer_bias = model.bias_matrix
for weights in layer_weight:
    print(weights)
print(layer_bias)

In [ ]:
print(wt.shape)